<a href="https://colab.research.google.com/github/vijayshankarrealdeal/intro_to_pytorch-Gans/blob/main/AUTOEn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data
from torch.autograd import Variable
import zipfile

In [5]:
with zipfile.ZipFile('/content/ml-100k.zip') as zip_ref:
  zip_ref.extractall('/content/data')

In [7]:
training_set = pd.read_csv('/content/data/ml-100k/u1.base',sep = '\t')
test_set = pd.read_csv('/content/data/ml-100k/u1.test',sep = '\t')

In [8]:
training_set = np.array(training_set,dtype='int')
test_set = np.array(test_set,dtype='int')

In [11]:
nb_users = max( max(training_set[:,0]),max(test_set[:,0]))
nb_movies = max( max(training_set[:,1]) , max(test_set[:,1]))
nb_users,nb_movies

(943, 1682)

In [16]:
def convert(data):
  new_data = []
  for user_id in range(1,nb_users+1):
    id_movie = data[:,1][data[:,0] == user_id]
    id_ratings = data[:,2][data[:,0] == user_id]
    new_zer = np.zeros(nb_movies)
    new_zer[id_movie - 1] = id_ratings
    id_ratings = new_zer
    new_data.append(list(id_ratings))
  return new_data

In [17]:
training_set = convert(training_set)
test_set = convert(test_set)

In [18]:
training_set = torch.FloatTensor(training_set)
test_set  = torch.FloatTensor(test_set)

In [28]:
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 20)
    self.fc2 = nn.Linear(20, 10)
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20, nb_movies)
    self.activation = nn.Sigmoid()
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x

In [33]:
sae = SAE()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.RMSprop(sae.parameters(),lr = 0.01,weight_decay=0.5)

In [34]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'\tloss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7717)
epoch: 2loss: tensor(1.0968)
epoch: 3loss: tensor(1.0535)
epoch: 4loss: tensor(1.0382)
epoch: 5loss: tensor(1.0309)
epoch: 6loss: tensor(1.0265)
epoch: 7loss: tensor(1.0238)
epoch: 8loss: tensor(1.0218)
epoch: 9loss: tensor(1.0207)
epoch: 10loss: tensor(1.0197)
epoch: 11loss: tensor(1.0189)
epoch: 12loss: tensor(1.0182)
epoch: 13loss: tensor(1.0179)
epoch: 14loss: tensor(1.0174)
epoch: 15loss: tensor(1.0173)
epoch: 16loss: tensor(1.0170)
epoch: 17loss: tensor(1.0167)
epoch: 18loss: tensor(1.0165)
epoch: 19loss: tensor(1.0165)
epoch: 20loss: tensor(1.0163)
epoch: 21loss: tensor(1.0160)
epoch: 22loss: tensor(1.0159)
epoch: 23loss: tensor(1.0158)
epoch: 24loss: tensor(1.0154)
epoch: 25loss: tensor(1.0157)
epoch: 26loss: tensor(1.0154)
epoch: 27loss: tensor(1.0153)
epoch: 28loss: tensor(1.0150)
epoch: 29loss: tensor(1.0132)
epoch: 30loss: tensor(1.0113)
epoch: 31loss: tensor(1.0093)
epoch: 32loss: tensor(1.0090)
epoch: 33loss: tensor(1.0045)
epoch: 34loss: tens

In [38]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9548)
